In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
import h5py
import seaborn as sns
import pandas as pd
from scipy.stats import pearsonr
import configs

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from models.networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset

# Parameters

In [2]:
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/'
activations_dir = f'{engram_dir}activations_pnet_all/'
pickles_dir = f'{engram_dir}pickles/'

In [3]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')

Device: cpu


# Save correlations with shuffles/mean-subtraction

In [4]:
file_prefix = 'repr_pearsonr'
shuff_file_prefix = 'shuffle_pearsonr'
bgs = ['pinkNoise', 'AudScene', 'Babble8Spkr']
snrs = [-9.0, -6.0, -3.0, 0.0, 3.0]

In [5]:
layers = ['conv1', 'conv2', 'conv3', 'conv4_W', 'conv5_W']

In [6]:
popln = {}
popln['corr'] = []
popln['corr_norm'] = []
popln['corr_mean_sub'] = []
popln['corr_norm_mean_sub'] = []
popln['timestep'] = []
popln['layer'] = []
popln['bg'] = []
popln['snr'] = []

for bg in bgs:
    for snr in snrs:
        print(f'{bg}, SNR {snr}')        
        with open(f'{pickles_dir}{file_prefix}_{bg}_snr{snr}.p', 'rb') as f:
            results = pickle.load(f)
            
        # Load layer shuffle results
        with open(f'{pickles_dir}{shuff_file_prefix}_{bg}_snr{snr}.p', 'rb') as f:
            shuff_results = pickle.load(f)
        
        popln_shuff = {}
        for l in layers:
            popln_shuff[l] = np.nanmean(np.array(
                shuff_results['popln_shuffle'])[
                np.array(shuff_results['popln_layer'])==l
                ])
            
        # Calculate popln normalized and un-normalized correlations
        for i in range(len(results['popln_layer'])):
            layer = results['popln_layer'][i]
            if layer not in layers: continue
            corr = results['popln_shuffle'][i]
            corr_norm = corr/popln_shuff[layer]
            popln['corr'].append(corr)
            popln['corr_norm'].append(corr_norm)
            popln['timestep'].append(results['popln_timestep'][i])
            popln['layer'].append(layer)
            popln['bg'].append(bg)
            popln['snr'].append(snr)
        
# Calculate popln mean subtractions
popln_mean = {}
popln_norm_mean = {}
for l in layers:
    popln_mean[l] = np.nanmean(np.array(
        popln['corr'])[np.logical_and(
        np.array(popln['layer'])==l,
        np.array(popln['timestep'])==0
        )])
    popln_norm_mean[l] = np.nanmean(np.array(
        popln['corr_norm'])[np.logical_and(
        np.array(popln['layer'])==l,
        np.array(popln['timestep'])==0
        )])

for corr, corr_norm, layer in zip(popln['corr'], popln['corr_norm'], popln['layer']):
    popln['corr_mean_sub'].append(corr - popln_mean[layer])
    popln['corr_norm_mean_sub'].append(corr_norm - popln_norm_mean[layer])

pinkNoise, SNR -9.0
pinkNoise, SNR -6.0
pinkNoise, SNR -3.0
pinkNoise, SNR 0.0
pinkNoise, SNR 3.0
AudScene, SNR -9.0
AudScene, SNR -6.0
AudScene, SNR -3.0
AudScene, SNR 0.0
AudScene, SNR 3.0
Babble8Spkr, SNR -9.0
Babble8Spkr, SNR -6.0
Babble8Spkr, SNR -3.0
Babble8Spkr, SNR 0.0
Babble8Spkr, SNR 3.0


In [7]:
with open(f'{pickles_dir}{file_prefix}_popln_collected.p', 'wb') as f:
    pickle.dump(popln, f)